In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from admm_approximate_solver import Approximate_ReLU_ADMM_Solver
from relu_utils import squared_loss, classifcation_accuracy

In [2]:
# generate toy data trying to fit noisy data to cosine
data = pd.read_csv("test_data/wineQuality-red.csv", delimiter=";")
X = np.array(data)[:,:11]
y = np.array(data.quality)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_train, d = X_train.shape
n_test = X_test.shape[0]

print(f"n train = {n_train}")
print(f"n test = {n_test}")

m = 10

n train = 1279
n test = 320


In [3]:
# TODO: solve simple pytorch problem

MLP = nn.Sequential(nn.Linear(d, m),
                    nn.ReLU(),
                    nn.Linear(m, np.max(y)+1))

print(MLP)

Sequential(
  (0): Linear(in_features=11, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=9, bias=True)
)


Solver doesn't converge well... Things needing fixing right now: 

- How to choose m, P_S?
- How to choose parameters $\rho$, step=$\gamma_{\alpha}$, $\beta$? 
- Converting from optimal weights $v, w$ to $u, \alpha$. Currently loss during training is consistently high but gets lower when calling .predict
- Runtime warning - divide by zero in optimizer. need to figure out why / fix

In [13]:
# solve cvx problem

solver = Approximate_ReLU_ADMM_Solver(m=10,
                                      P_S=10,
                                      rho=0.0001,
                                      step=0.00001,
                                      beta=0.0001,
                                      bias=True,
                                      loss_func=squared_loss,
                                      acc_func=classifcation_accuracy)

solver.optimize(X_train, y_train, max_iter=100, verbose=True)


print("CONVEX PROBLEM WEIGHTS:")
y_hat_train = solver.predict(X_train, weights="C-ReLU")
y_hat_test = solver.predict(X_test, weights="C-ReLU")
print(f"Train loss: {squared_loss(y_hat_train, y_train)}")
print(f"Train accuracy: {classifcation_accuracy(y_hat_train, y_train)}")
print(f"Test loss: {squared_loss(y_hat_test, y_test)}")
print(f"Test accuracy: {classifcation_accuracy(y_hat_test, y_test)}")

print("NONCONVEX PROBLEM WEIGHTS:")
y_hat_train = solver.predict(X_train, weights="NC-ReLU")
y_hat_test = solver.predict(X_test, weights="NC-ReLU")
print(f"Train loss: {squared_loss(y_hat_train, y_train)}")
print(f"Train accuracy: {classifcation_accuracy(y_hat_train, y_train)}")
print(f"Test loss: {squared_loss(y_hat_test, y_test)}")
print(f"Test accuracy: {classifcation_accuracy(y_hat_test, y_test)}")

iter = 0, loss = 20642.500000000004, acc = 0.0
iter = 1, loss = 252.12420344532072, acc = 0.617670054730258
iter = 2, loss = 251.9245329826125, acc = 0.6215793588741204
iter = 3, loss = 251.8436917007559, acc = 0.6215793588741204
iter = 4, loss = 251.81006738670993, acc = 0.6207974980453479
iter = 5, loss = 251.79552457465113, acc = 0.6200156372165755
iter = 6, loss = 251.7888480383544, acc = 0.6200156372165755
iter = 7, loss = 251.78550294310807, acc = 0.6200156372165755
iter = 8, loss = 251.78362337074176, acc = 0.6200156372165755
iter = 9, loss = 251.7824238757141, acc = 0.6200156372165755
iter = 10, loss = 251.78156410041814, acc = 0.6200156372165755
iter = 11, loss = 251.78089012526422, acc = 0.6200156372165755
iter = 12, loss = 251.78032824031698, acc = 0.6200156372165755
iter = 13, loss = 251.77984071058057, acc = 0.6200156372165755
iter = 14, loss = 251.7794068266156, acc = 0.6200156372165755
iter = 15, loss = 251.77901437109787, acc = 0.6200156372165755
iter = 16, loss = 251.7